In [1]:
import sys
import deepHSI

from pathlib import Path

import numpy as np
from lightning.pytorch import Trainer, seed_everything

# Custom module imports
from deepHSI.datamodule.components import HyperspectralDataset
from deepHSI.datamodule.components.utils import *

from deepHSI.datamodule.medical_datasets import BloodDetectionHSIDataModule

from deepHSI.datamodule.remote_sensing_datasets import *
# from deepHSI.datamodule.remote_sensing_datasets.paviaC import PaviaCDataModule

from deepHSI.models.architectures import HSIFCModel, \
    SpectralSpatialCNN, HyperspectralCNNDetector
from deepHSI.models.task_algos import HSIClassificationModule

seed_everything(42, workers=True)

# Importing from `lightning` instead of `pytorch_lightning`
import lightning as L

# PyTorch and metrics imports
import torch
from torchmetrics import F1Score, Precision, Recall

# from lightning import Trainer

torch.set_float32_matmul_precision("medium")

Seed set to 42


In [2]:
from pathlib import Path

# Specify the directory to save checkpoints
ckpt_dir = Path("/home/sayem/Desktop/deepHSI/notebooks/ckpt")

# Function to clear directory

def clear_directory(path: Path):
    if path.is_dir():
        for item in path.iterdir():
            if item.is_dir():
                clear_directory(item)
                item.rmdir()
            else:
                item.unlink()
    else:
        path.mkdir(parents=True, exist_ok=True)


# Clear and/or create the log and checkpoint directories
clear_directory(ckpt_dir)

In [3]:
from deepHSI.models.task_algos import BaseModule

In [4]:
import torch
from torch.optim import Adam, lr_scheduler

# Define the parameters for the optimizer
optimizer_params = {
    'lr': 0.001,
    'betas': (0.9, 0.999),
    'eps': 1e-08,
    'weight_decay': 0,
}

# Define the parameters for the scheduler
scheduler_params = {
    'step_size': 10,
    'gamma': 0.1,
}

In [5]:
# Define the parameters for the data module
data_dir = "/home/sayem/Desktop/deepHSI/data"  # Specify the directory where you want the data to be downloaded

# Include 'batch_size', 'num_workers', and 'num_classes' within the hyperparams dictionary
hyperparams = {
    "batch_size": 64,
    "num_workers": 24,
    "patch_size": 10,
    "center_pixel": True,
    "supervision": "full",
    "num_classes": 10,  # Define the number of classes in your dataset
}

# Assuming YourModel is defined elsewhere and num_classes is known
input_channels = 102

# Define custom metrics for the classification task using the updated hyperparams
custom_metrics = {
    "precision": Precision(
        num_classes=hyperparams["num_classes"], average="macro", task="multiclass"
    ),
    "recall": Recall(num_classes=hyperparams["num_classes"], average="macro", task="multiclass"),
    "f1": F1Score(num_classes=hyperparams["num_classes"], average="macro", task="multiclass"),
}

In [6]:
from deepHSI.datamodule.transforms.hsi_transforms import *

# Instantiate individual transformations
normalize_transform = HSINormalize()
flip_transform = HSIFlip()
rotate_transform = HSIRotate()
noise_transform = HSISpectralNoise(mean=0.0, std=0.01)
shift_transform = HSISpectralShift(shift=2)
drop_transform = HSIRandomSpectralDrop(drop_prob=0.1)

# Compose transformations together
composed_transform = Compose([
    normalize_transform,
    flip_transform,
    # rotate_transform,
    noise_transform,
    shift_transform,
    drop_transform
])

In [7]:
# # # Initialize the PyTorch Lightning Trainer
# # trainer = Trainer(max_epochs=10, precision='16-mixed', accelerator='gpu', devices=1)
max_epochs = 200

# Initialize the PaviaCDataModule with the updated arguments
pavia_c_datamodule = PaviaCDataModule(
    data_dir=data_dir, hyperparams=hyperparams, \
    transform=composed_transform,  # Pass hyperparams which now includes num_classes
)

pavia_c_datamodule.prepare_data()
pavia_c_datamodule.setup()

Dataset 'PaviaC' already exists. Skipping download.


In [8]:
batch = next(iter(pavia_c_datamodule.train_dataloader()))

In [9]:
batch[0].shape

torch.Size([64, 1, 102, 10, 10])

## Test vae

In [10]:
from torch import nn

class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(Encoder, self).__init__()
        # Convolutional layers
        self.conv_layers = nn.Sequential(
            nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        self.flatten_size = 64 * 13 * 2 * 2  # Adjust based on the output size of the last conv layer
        # Fully connected layers for producing mean and log variance
        self.fc_mu = nn.Linear(in_features=self.flatten_size, out_features=latent_dim)
        self.fc_log_var = nn.Linear(in_features=self.flatten_size, out_features=latent_dim)

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        mu = self.fc_mu(x)
        log_var = self.fc_log_var(x)
        return torch.cat((mu, log_var), dim=1)  # Concatenate mu and log_var along the feature dimension


class Decoder(nn.Module):
    def __init__(self, latent_dim):
        super(Decoder, self).__init__()
        # The initial fully connected layer
        self.fc = nn.Linear(in_features=latent_dim, out_features=64 * 13 * 2 * 2)
        # Transpose convolutional layers
        self.conv_transpose_layers = nn.Sequential(
            nn.ConvTranspose3d(in_channels=64, out_channels=32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose3d(in_channels=32, out_channels=16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose3d(in_channels=16, out_channels=1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 64, 13, 2, 2)  # Reshape to match the expected input shape for the transpose conv layers
        x = self.conv_transpose_layers(x)
        return x

In [11]:
from deepHSI.models.task_algos import VAEModule

In [12]:
latent_dim = 20

vae = VAEModule(encoder=Encoder, decoder=Decoder, \
    latent_dim=latent_dim, optimizer_constructor=Adam,
    optimizer_params=optimizer_params,
    scheduler_constructor=lr_scheduler.StepLR,
    scheduler_params=scheduler_params,)

In [21]:
vae.train(batch)

ValueError: training mode is expected to be boolean

In [14]:
import os
import wandb
from pytorch_lightning.loggers import WandbLogger

# Assuming the BaseModule, optimizer, and scheduler are initialized as follows:
# module = BaseModule(...)

# Extract the optimizer and scheduler names
optimizer_id = vae.optimizer_constructor.__name__  # Assuming optimizer_constructor is stored in the VAEModule
scheduler_id = 'None' if vae.scheduler_constructor is None else vae.scheduler_constructor.__name__

# Dynamically get the model name - since VAEModule doesn't directly have a 'model' attribute, we use its class name
model_name = vae.__class__.__name__  # Using the class name of the VAEModule itself


# Set the WandB notebook name for better organization
os.environ["WANDB_NOTEBOOK_NAME"] = "1.0-hsi-initial-data-exploration.ipynb"
wandb.login()

# Dynamically construct the run name to include key experiment details
dynamic_run_name = f"{optimizer_id}-with-{scheduler_id}-{model_name}"

# Extend the dynamic tags to include other important aspects of the experiment
dynamic_tags = [
    optimizer_id, scheduler_id, model_name,
    # Add more tags as needed, e.g., dataset name, special model configurations
]

print(dynamic_tags)

# Include additional notes for the experiment to describe unique aspects or goals
experiment_notes = f"Testing {optimizer_id} optimizer with \
    {scheduler_id} scheduler on {model_name}."

# Initialize WandbLogger with dynamic configurations
wandb_logger = WandbLogger(
    name=dynamic_run_name,
    project="PaviaC",
    save_dir="/home/sayem/Desktop/deepHSI/notebooks/wandb",
    offline=False,
    tags=dynamic_tags,
    notes=experiment_notes,  # Add the experiment notes
)

# # Assume `hyperparams` is a dictionary containing your experiment-specific hyperparameters
# # Include additional hyperparameters related to the optimizer and scheduler
# hyperparams = {
#     "learning_rate": 0.001,  # Example hyperparameter
#     # Include other experiment-specific hyperparameters
# }

# Update the WandB config with a comprehensive set of experiment details
wandb_logger.experiment.config.update({
    "optimizer": optimizer_id,
    "scheduler": scheduler_id,
    # "model": model_name,
    **hyperparams,
})

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find 1.0-hsi-initial-data-exploration.ipynb.
wandb: Currently logged in as: k61. Use `wandb login --relogin` to force relogin
wandb: WARNING Path /home/sayem/Desktop/deepHSI/notebooks/wandb/wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path /home/sayem/Desktop/deepHSI/notebooks/wandb/wandb/ wasn't writable, using system temp directory


['Adam', 'StepLR', 'VAEModule']


In [16]:
# Callbacks: Pytorch Callbacks: See L doc
# Define the EarlyStopping callback
early_stop_callback = L.pytorch.callbacks.EarlyStopping(
    monitor="val/f1",  # Specify the metric to monitor
    patience=20,  # Number of epochs with no improvement after which training will be stopped
    verbose=True,  # Whether to print logs to stdout
    mode="max",  # In 'min' mode, training will stop when the quantity monitored has stopped decreasing
    check_on_train_epoch_end=False,
)

# from lightning.pytorch.callbacks import ModelCheckpoint

# Define the ModelCheckpoint callback
model_checkpoint = L.pytorch.callbacks.ModelCheckpoint(
    monitor="val/f1",  # Metric to monitor
    dirpath=str(ckpt_dir),  # Convert Path object to string, Directory to save checkpoints
    filename="best-checkpoint-{epoch:02d}-{val/f1:.2f}",  # Checkpoint file name
    save_top_k=1,  # Save only the best checkpoint
    mode="max",  # 'max' because we want to maximize 'val/f1'
    verbose=True,  # Print a message when a new best is found
    auto_insert_metric_name=False,  # Prevents metric names being inserted into filename automatically
)

rich_pbar_callback = L.pytorch.callbacks.RichProgressBar(
    refresh_rate=1,
    leave=True,
)

lr_monitor_callback = L.pytorch.callbacks.LearningRateMonitor(logging_interval='epoch') 

In [17]:
# Initialize the PyTorch Lightning Trainer with fast_dev_run enabled
trainer = L.Trainer(
    fast_dev_run=False,  # Enable fast_dev_run
    precision="16-mixed",  # Use 16-bit precision
    accelerator="auto",  # Specify the accelerator as GPU
    max_epochs=max_epochs,
    log_every_n_steps=3,
    callbacks=[
        # lr_finder_callback,
        early_stop_callback,
        model_checkpoint,
        # confusion_matrix_callback,
        # batch_finder_callback,
        lr_monitor_callback,
        # model_weight_logger_callback,
    ],  # rich_pbar_callback],
    logger=wandb_logger,
    deterministic=False,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
trainer.fit(vae, datamodule=pavia_c_datamodule)

/home/sayem/anaconda3/envs/deepHSI/lib/python3.12/site-packages/lightning/pytorch/trainer/configuration_validator.py:72: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.


Dataset 'PaviaC' already exists. Skipping download.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type          | Params
---------------------------------------
0 | vae  | VAEPyroModule | 342 K 
---------------------------------------
342 K     Trainable params
0         Non-trainable params
342 K     Total params
1.369     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

ValueError: Shape mismatch inside plate('data') at site obs dim -1, 64 vs 16
                           Trace Shapes:                     
                            Param Sites:                     
                     decoder$$$fc.weight         3328   20   
                       decoder$$$fc.bias              3328   
decoder$$$conv_transpose_layers.0.weight 64 32 3    3    3   
  decoder$$$conv_transpose_layers.0.bias                32   
decoder$$$conv_transpose_layers.2.weight 32 16 3    3    3   
  decoder$$$conv_transpose_layers.2.bias                16   
decoder$$$conv_transpose_layers.4.weight 16  1 3    3    3   
  decoder$$$conv_transpose_layers.4.bias                 1   
                           Sample Sites:                     
                               data dist                 |   
                                   value           64    |   
                             latent dist           64    | 20
                                   value           64  

In [ ]:
# Fit the model using the train dataset from the data module
dictionary = trainer.test(hsi_classifier, pavia_c_datamodule, verbose=True)
# trainer.fit(hsi_module, datamodule=pavia_c_datamodule)tree
# Use train_dataloader() instead of train_dataset

In [ ]:
# After training or experiments are done, finish the WandB run
wandb.finish()